# Load fact_order_items từ Silver lên Gold

## Mục tiêu
- Load dữ liệu từ Silver.fact_order_items → Gold.fact_order_items
- Không cần cleaning, chỉ load trực tiếp
- Đảm bảo data quality cho Gold layer

## Quy trình 3 bước
1. **Load from Silver** - Đọc dữ liệu từ Silver database
2. **Load to Gold** - Load vào Gold database
3. **Create Dictionary** - Tạo data dictionary


## Bước 1: Import và Setup


In [4]:
# Import các thư viện cần thiết
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from dotenv import load_dotenv
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Load biến môi trường từ file .env
load_dotenv()

# Lấy thông tin kết nối database
DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_SILVER = os.getenv("DB_SILVER")
DB_GOLD = os.getenv("DB_GOLD")

# Tạo kết nối tới Silver và Gold database
silver_engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_SILVER}")
gold_engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_GOLD}")

print("Đã kết nối Silver và Gold database thành công")
print(f"Silver DB: {DB_SILVER}")
print(f"Gold DB: {DB_GOLD}")
print(f"Thời gian bắt đầu: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


Đã kết nối Silver và Gold database thành công
Silver DB: winner_silver
Gold DB: winner_gold
Thời gian bắt đầu: 2025-10-18 00:05:27


## Bước 2: Load dữ liệu từ Silver


In [5]:
# Load dữ liệu fact_order_items từ Silver database
print("=== LOADING FACT_ORDER_ITEMS FROM SILVER ===")

# Load toàn bộ bảng fact_order_items
order_items_df = pd.read_sql("SELECT * FROM fact_order_items", silver_engine)

# Hiển thị thông tin cơ bản về dataset
print(f"Shape: {order_items_df.shape}")
print(f"Columns: {list(order_items_df.columns)}")
print(f"Memory usage: {order_items_df.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")

# Hiển thị sample data
print("\n=== SAMPLE DATA ===")
print(order_items_df.head(3))

# Hiển thị data types
print("\n=== DATA TYPES ===")
print(order_items_df.dtypes)

# Kiểm tra null values
print("\n=== NULL VALUES CHECK ===")
null_summary = order_items_df.isnull().sum()
print("Null values per column:")
for col, null_count in null_summary.items():
    if null_count > 0:
        print(f"  {col}: {null_count} ({null_count/len(order_items_df)*100:.1f}%)")
    else:
        print(f"  {col}: {null_count} (0.0%)")


=== LOADING FACT_ORDER_ITEMS FROM SILVER ===
Shape: (46611, 11)
Columns: ['order_item_id', 'order_id', 'product_id', 'variation_id', 'item_quantity', 'item_price', 'item_discount', 'return_quantity', 'returned_count', 'product_name', 'product_detail']
Memory usage: 22.48 MB

=== SAMPLE DATA ===
  order_item_id order_id                            product_id  \
0  ITEM_40458_0    40458  358156d8-2777-4aab-b584-a26813fb4b95   
1  ITEM_40455_0    40455  ea5b416d-511e-47d9-841a-957fb22e68a8   
2  ITEM_40454_0    40454  ea5b416d-511e-47d9-841a-957fb22e68a8   

                           variation_id  item_quantity  item_price  \
0  847cf093-8f90-4f01-85ae-800ec31432a1              1    629000.0   
1  4e1af645-e80d-4e74-bcd1-e23c296999cf              1    649000.0   
2  46ae8aed-2296-4f5e-9665-39fd2203be6a              1    649000.0   

   item_discount  return_quantity  returned_count      product_name  \
0              0                0               0  Áo thu đông.W031   
1              0

## Bước 3: Load vào Gold Database


In [6]:
# Load vào Gold Database
print("=== LOADING TO GOLD DATABASE ===")

from sqlalchemy.types import BigInteger, Integer, String, Text, DateTime, Float

# Định nghĩa schema mapping cho MySQL
dtype_mapping = {}

# ID fields - String (để xử lý mixed data types)
id_fields = ["order_item_id", "order_id", "product_id", "variation_id"]
for field in id_fields:
    if field in order_items_df.columns:
        dtype_mapping[field] = String(100)

# String fields - VARCHAR(255)
string_fields = ["product_name", "product_detail"]
for field in string_fields:
    if field in order_items_df.columns:
        dtype_mapping[field] = String(255)

# Numeric fields
float_fields = ["item_price", "item_discount"]
for field in float_fields:
    if field in order_items_df.columns:
        dtype_mapping[field] = Float()

# Integer fields
int_fields = ["item_quantity", "return_quantity", "returned_count"]
for field in int_fields:
    if field in order_items_df.columns:
        dtype_mapping[field] = Integer()

# DateTime fields
datetime_fields = ["created_at", "updated_at"]
for field in datetime_fields:
    if field in order_items_df.columns:
        dtype_mapping[field] = DateTime()

print(f"Schema mapping defined for {len(dtype_mapping)} columns")

# Load vào Gold database
table_name = "fact_order_items"

try:
    order_items_df.to_sql(
        table_name,
        con=gold_engine,
        if_exists="replace",  # Ghi đè dữ liệu cũ
        index=False,
        dtype=dtype_mapping
    )
    
    print(f"Đã load {order_items_df.shape[0]} records vào Gold: {table_name}")
    
    # Verify load
    verification_query = f"SELECT COUNT(*) as count FROM {table_name}"
    verification_result = pd.read_sql(verification_query, gold_engine)
    loaded_count = verification_result['count'].iloc[0]
    
    print(f"Verification: {loaded_count} records trong database")
    print(f"Schema: {len(dtype_mapping)} columns với explicit typing")
    
    # Sample data từ database
    sample_query = f"SELECT * FROM {table_name} LIMIT 5"
    sample_db = pd.read_sql(sample_query, gold_engine)
    print(f"\nSample data từ Gold database:\n{sample_db}")
    
except Exception as e:
    print(f"Error loading to Gold: {str(e)}")
    raise

print(f"\nLoad fact_order_items completed successfully!")
print(f"Ready for EDA and analytics")


=== LOADING TO GOLD DATABASE ===
Schema mapping defined for 11 columns
Đã load 46611 records vào Gold: fact_order_items
Verification: 46611 records trong database
Schema: 11 columns với explicit typing

Sample data từ Gold database:
  order_item_id order_id                            product_id  \
0  ITEM_40458_0    40458  358156d8-2777-4aab-b584-a26813fb4b95   
1  ITEM_40455_0    40455  ea5b416d-511e-47d9-841a-957fb22e68a8   
2  ITEM_40454_0    40454  ea5b416d-511e-47d9-841a-957fb22e68a8   
3  ITEM_40453_0    40453  9764f7be-e5d1-4c68-92fe-067b2437342b   
4  ITEM_40452_0    40452  ea5b416d-511e-47d9-841a-957fb22e68a8   

                           variation_id  item_quantity  item_price  \
0  847cf093-8f90-4f01-85ae-800ec31432a1              1    629000.0   
1  4e1af645-e80d-4e74-bcd1-e23c296999cf              1    649000.0   
2  46ae8aed-2296-4f5e-9665-39fd2203be6a              1    649000.0   
3  06573beb-4ce8-4585-b4c9-47a426fbbab1              1    649000.0   
4  a7c57d7d-9e04-4f4

## Bước 4: Tạo Data Dictionary cho Gold


In [7]:
# Tạo Data Dictionary cho Gold
print("=== CREATING GOLD DATA DICTIONARY ===")

def get_business_meaning_vietnamese(column_name):
    """Get business meaning for each column in Vietnamese"""
    business_meanings = {
        # Định danh & Khóa
        "order_item_id": "Mã định danh item trong đơn hàng (khóa chính)",
        "order_id": "Mã đơn hàng (khóa ngoại đến fact_orders)",
        "product_id": "Mã sản phẩm (khóa ngoại đến dim_products)",
        
        # Thông tin sản phẩm
        "product_name": "Tên sản phẩm trong đơn hàng",
        "variant_name": "Tên biến thể sản phẩm (màu sắc, kích thước)",
        "variant_value": "Giá trị biến thể cụ thể",
        
        # Giá và số lượng
        "price": "Giá đơn vị sản phẩm (VND)",
        "quantity": "Số lượng sản phẩm trong đơn hàng",
        "total_amount": "Tổng tiền của item (price × quantity)",
        "discount_amount": "Số tiền giảm giá cho item này",
        
        # Thời gian
        "created_at": "Thời gian tạo bản ghi item",
        "updated_at": "Thời gian cập nhật bản ghi cuối cùng"
    }
    return business_meanings.get(column_name, "Chưa định nghĩa ý nghĩa business")

# Tạo Data Dictionary
dict_data = []
for col in order_items_df.columns:
    col_info = {
        "table_name": "fact_order_items",
        "column_name": col,
        "dtype": str(order_items_df[col].dtype),
        "sql_type": str(dtype_mapping.get(col, "Not defined")),
        "null_count": order_items_df[col].isnull().sum(),
        "null_pct": round(order_items_df[col].isnull().mean() * 100, 2),
        "unique_count": order_items_df[col].nunique(),
        "sample_values": str(order_items_df[col].dropna().unique()[:3].tolist()),
        "business_meaning": get_business_meaning_vietnamese(col),
        "extraction_date": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    dict_data.append(col_info)

data_dictionary = pd.DataFrame(dict_data)

# Hiển thị Data Dictionary
print(f"Generated Data Dictionary for {len(data_dictionary)} columns")
print("\n=== DATA DICTIONARY ===")
print(data_dictionary)

# Lưu vào file Excel
excel_path = "Technical_Document/Gold_dictionary.xlsx"

try:
    from openpyxl import load_workbook
    
    # Kiểm tra file Excel có tồn tại không
    try:
        # Load workbook hiện tại
        wb = load_workbook(excel_path)
        
        # Lấy sheet đầu tiên
        ws = wb.active
        
        # Kiểm tra xem có dữ liệu cũ không
        if ws.max_row > 1:
            print(f"Found existing data in {excel_path}, appending new data...")
        else:
            print(f"File {excel_path} exists but is empty, adding header and data...")
            
    except FileNotFoundError:
        print(f"File {excel_path} not found, creating new file...")
        wb = None
    except Exception as e:
        print(f"Error loading {excel_path}: {str(e)}, creating new file...")
        wb = None
    
    if wb is None:
        # Tạo file mới với header
        data_dictionary.to_excel(excel_path, index=False, sheet_name='Gold_Dictionary')
        print(f"✅ Created new file: {excel_path}")
    else:
        # Append vào file hiện tại
        from openpyxl.utils.dataframe import dataframe_to_rows
        
        # Tìm dòng cuối cùng có dữ liệu
        last_row = ws.max_row
        
        # Thêm dữ liệu mới từ dòng tiếp theo
        for r in dataframe_to_rows(data_dictionary, index=False, header=False):
            last_row += 1
            for c_idx, value in enumerate(r, 1):
                ws.cell(row=last_row, column=c_idx, value=value)
        
        # Lưu file
        wb.save(excel_path)
        print(f"✅ Appended {len(data_dictionary)} rows to: {excel_path}")
    
except Exception as e:
    print(f"❌ Error appending to Data Dictionary: {str(e)}")
    # Fallback: tạo file mới
    try:
        data_dictionary.to_excel(excel_path, index=False)
        print(f"✅ Created new file as fallback: {excel_path}")
    except Exception as e2:
        print(f"❌ Error creating fallback file: {str(e2)}")

# Summary Report
print(f"\n=== GOLD DICTIONARY SUMMARY ===")
print(f"Table: fact_order_items")
print(f"Total columns: {len(data_dictionary)}")
print(f"Data Dictionary: {excel_path}")
print(f"Created at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

print(f"\n=== SAMPLE DICTIONARY ENTRIES ===")
sample_dict = data_dictionary[['column_name', 'dtype', 'business_meaning']].head(5)
print(sample_dict)


=== CREATING GOLD DATA DICTIONARY ===
Generated Data Dictionary for 11 columns

=== DATA DICTIONARY ===
          table_name      column_name    dtype      sql_type  null_count  \
0   fact_order_items    order_item_id   object  VARCHAR(100)           0   
1   fact_order_items         order_id   object  VARCHAR(100)           0   
2   fact_order_items       product_id   object  VARCHAR(100)           0   
3   fact_order_items     variation_id   object  VARCHAR(100)           0   
4   fact_order_items    item_quantity    int64       INTEGER           0   
5   fact_order_items       item_price  float64         FLOAT           0   
6   fact_order_items    item_discount    int64         FLOAT           0   
7   fact_order_items  return_quantity    int64       INTEGER           0   
8   fact_order_items   returned_count    int64       INTEGER           0   
9   fact_order_items     product_name   object  VARCHAR(255)           0   
10  fact_order_items   product_detail   object  VARCHAR(255)